In [1]:
from mdp import generate_cliffworld, value_iteration, v_star
import numpy as np
state_space, action_space, rho_cf, P_cf, r_cf = generate_cliffworld()

# Matrix formulation of policy iteration and density iteration

now `P_Cliffworld` is a $21\times4\times21$ matrix. According to the basic policy evaluation, 
$$Q^\pi = r + \gamma P V$$
where $Q\in\mathbb R^{|\mathcal S||\mathcal A|}$. Then we know that P is in this formulation,
$$P\in\mathbb R^{|\mathcal S||\mathcal A|\times |\mathcal S|}$$, where $P_{ij}$ is the probability of take state-action pair $i$ (in a total of $n\times m$) and transition to next state $s'_j$.
where $$
P = \begin{bmatrix}
    P(s'\mid s_1, a_1)\\
    P(s'\mid s_1, a_2)\\
    \vdots\\
    P(s'\mid s_n, a_m)
\end{bmatrix}
$$

For the Q-version of the policy evaluation, we have

$$
Q^\pi = r + \gamma P\Pi Q
$$
here $\Pi\in\mathbb R^{|\mathcal S|\times|\mathcal S||\mathcal A|}$. We need to write the $\Pi$ in a block diagonal matrix,
$$
\Pi = \begin{bmatrix}
\pi(a\mid s = s_1) & 0 & \dots & 0\\
0 & \pi(a\mid s = s_2) & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \dots & \pi(a\mid s = s_n)
\end{bmatrix}
$$

Then if we do matrix multiplication of $P$ and $\Pi$, the results will be 
$$
P\Pi\in\mathbb R^{|\mathcal S||\mathcal A|\times|\mathcal S||\mathcal A|}
$$

In [2]:
# optimal policy
gamma = 0.9
K = 100
v_vi, pi_vi, gaps_vi = value_iteration(P_cf, r_cf, gamma, K, v_star(), theta=1e-4)
pi_vi

Converged at iteration 6


array([3., 0., 0., 0., 0., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1., 1.,
       1., 1., 2., 0.])

In [3]:
def make_p_mat(P):
    P_mat = np.zeros([len(state_space) * len(action_space), len(state_space)])
    for s in range(len(state_space)):
        for a in range(len(action_space)):
            P_mat[s * len(action_space) + a] = P[s, a]
    return P_mat
Pmat = make_p_mat(P_cf)

# Basics about matrix formulation

$$
V = \Pi Q
$$

- matrix of $P^\pi(s', a'\mid s, a ) = P\Pi$
- matrix of $P^\pi(s'\mid s) = \Pi P$

In [4]:
# 
def make_pi_mat(pi):
    pi_mat = np.zeros([len(state_space), len(state_space) * len(action_space)])
    for i in range(len(state_space)):
        action = pi[i]
        pi_mat[i, i * len(action_space) + int(action)] = 1
    return pi_mat
Pimat = make_pi_mat(pi_vi)

In [5]:
v_star_mat = np.linalg.inv(np.eye(Pmat.shape[0]) - gamma * Pmat @ Pimat) @ r_cf.reshape([-1, 1])
np.isclose((Pimat @ v_star_mat).squeeze(), v_star())

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

# density calculation

## enumerate version
According to the density recursive,
$$
d^\pi(s') = (1 - \gamma)\rho(s') + \sum_{(s, a)} P(s'\mid s, a)d(s)\pi(a\mid s)
$$

In [15]:
from copy import deepcopy
def density_evaluation(pi):
    density = deepcopy(rho_cf)
    converged = False
    iters = 0
    while not converged:
        d_last = deepcopy(density)
        for i in range(len(state_space)):
            density[i] = (1 - gamma) * rho_cf[i]
            for j in range(len(state_space)):
                for k in range(len(action_space)):
                    density[i] += gamma * P_cf[j, k, i] * d_last[j] * (pi[j] == k)
        if np.linalg.norm(density - d_last) < 1e-5:
            converged = True
        iters += 1
    print(f"converged at iteration {iters}, density sums up to {np.sum(density)}")
    return density
d_pi_vi = density_evaluation(pi_vi)

converged at iteration 9, density sums up to 1.0


## matrix version
$d^\pi \in \mathbb R^{|\mathcal S||\mathcal A|} = \Pi^\top d$

$$
d^\pi = (1-\gamma)\rho + \gamma P^\top d^\pi= (1-\gamma)\rho + \gamma P^\top \Pi^\top d
$$

Therefore, 
$$
d = (I - \gamma (\Pi P)^\top)^{-1} (1-\gamma) \rho
$$

In [20]:
d_pi_vi_mat = np.linalg.solve(np.eye(len(state_space)) - gamma * (Pimat @ Pmat).transpose(), (1 - gamma) * rho_cf)
np.isclose(d_pi_vi, d_pi_vi_mat)
# d_pi_vi_mat

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

## state-action version
$$
\Pi^\top d = (1-\gamma)\Pi^\top \rho + \gamma \Pi^\top P^\top \Pi^\top d
$$
$$
d^\pi = (1-\gamma)\Pi^\top \rho + \gamma \Pi^\top P^\top d^\pi
$$
$$
d^\pi = (1-\gamma)\Pi^\top \rho + \gamma (P\Pi)^\top d^\pi
$$